<h1>What are the lyrical differences between pop, rock and hip-hop songs? </h1>
<ul> 
    <li>
        <p>
            What are the common themes in each genre, are they shared or do they differentiate from genre to genre
        </p>
    </li>
        <li>
        <p>
            Do certain genres contain more mature language than the others?
        </p>
    </li>
</ul>

In [2]:
import pandas

In [3]:
lyrics = pandas.read_csv('lyrics-data.csv')
english_songs = lyrics['Idiom'] =='ENGLISH' 


In [4]:
english = lyrics[english_songs]

In [5]:
artist = pandas.read_csv('artists-data.csv')
print(artist.shape)
pop = artist['Genre'] == 'Pop'
rock = artist['Genre'] == 'Rock'
hiphop = artist['Genre'] == 'Hip Hop'

popsongs = artist[pop]
rocksongs = artist[rock]
hiphopsongs = artist[hiphop]

english_popsongs = pandas.merge(popsongs, english, left_on='Link', right_on='ALink')
english_popsongs.to_csv('englishPopSongs.csv', index=False)

english_rocksongs = pandas.merge(rocksongs, english, left_on='Link', right_on='ALink')
english_rocksongs.to_csv('englishRockSongs.csv', index=False)

english_hiphopsongs = pandas.merge(hiphopsongs, english, left_on='Link', right_on='ALink')
english_hiphopsongs.to_csv('englishHipHopSongs.csv', index=False)



(3242, 6)


In [6]:

# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [7]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [8]:
# Apply a third round of cleaning
def clean_text_round3(text):
    '''Get rid of â€™Ă©Ł§ˇŞ '''
    text = re.sub('[^A-Za-z0-9 ]+', '',text)
    return text

round3 = lambda x: clean_text_round3(x)

In [9]:
data_clean_round1 = pandas.DataFrame(english_popsongs['Lyric'].apply(clean_text_round1))

In [10]:
data_clean_round2 = pandas.DataFrame(data_clean_round1['Lyric'].apply(clean_text_round2))

In [11]:
data_clean_round3 = pandas.DataFrame(data_clean_round2['Lyric'].apply(clean_text_round3))

In [12]:
data_clean_round3.to_csv('round3.csv', index=False)

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [34]:
# remove stop words
def clean_text_round4(text):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text)
    cleanedText = '';
    for word in words:
        if word not in stop_words:
            cleanedText += '{} '.format(word)
    return cleanedText

In [35]:
data_clean_round4 = pandas.DataFrame(data_clean_round3['Lyric'].apply(clean_text_round4))

In [36]:
data_clean_round4.to_csv('round4.csv', index=False)

In [19]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{'doesn', 'through', 'weren', 'why', 'needn', 'shan', 'mightn', 'shouldn', 'can', 'ain', 'ourselves', 'over', "couldn't", 'wouldn', 'above', 'has', 'all', 'am', 'yourself', "needn't", 'i', 'ours', 'have', 'further', 'couldn', 'ma', 'didn', 'only', 'same', 'more', 'isn', "you're", 'some', 'which', 'against', 'him', "you'll", 'who', 'what', 'hers', 'himself', "aren't", 'as', 'or', 'theirs', 'here', 'myself', 'yours', 're', 'its', 'for', "should've", 'll', 'once', 'under', 'haven', 'mustn', 'where', 'won', 'we', 'these', "you've", 'nor', 'themselves', "you'd", "hadn't", 'out', 'after', 'any', 'his', 'into', 'don', 'both', 'an', 'not', 'from', 'their', 'be', 'such', 'during', 'and', "it's", "don't", 'but', "weren't", 'of', 'so', 'should', 'herself', "hasn't", 'other', "that'll", 'it', 'was', "didn't", 'too', 'were', 'whom', 'being', "isn't", 'hadn', 'her', 'below', 'o', 'they', 'been', 'do', 'our', 'did', 'while', "mightn't", 'he', 'this', 'up', 'before', 'to', 'that', "she's", 'just', 've